# Notebook 112: 適応学習率手法 ― Adagrad, RMSprop, Adam

## Adaptive Learning Rate Methods

---

### このノートブックの位置づけ

**Phase 10「最適化手法」** の第3章として、パラメータごとに学習率を自動調整する **適応学習率手法** を学びます。

### 学習目標

1. **Adagrad** の仕組みと問題点を理解する
2. **RMSprop** による改良を理解する
3. **Adam** の完全な理解と実装
4. **AdamW** と Weight Decay の違いを理解する

### 前提知識

- Notebook 110-111 の内容

---

## 目次

1. [適応学習率の必要性](#1-適応学習率の必要性)
2. [Adagrad](#2-adagrad)
3. [RMSprop](#3-rmsprop)
4. [Adam](#4-adam)
5. [AdamW と Weight Decay](#5-adamw-と-weight-decay)
6. [手法の比較](#6-手法の比較)
7. [演習問題](#7-演習問題)
8. [まとめと次のステップ](#8-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. 適応学習率の必要性

### 1.1 固定学習率の問題

SGDやMomentumでは、すべてのパラメータに同じ学習率を適用します。しかし：

- **頻繁に更新されるパラメータ**: 小さな学習率が望ましい
- **まれに更新されるパラメータ**: 大きな学習率が望ましい

例: 単語埋め込み（word embedding）では、頻出語と稀少語で勾配の規模が大きく異なる

In [ ]:
# パラメータごとの勾配規模の違い

# シミュレーション: 2つのパラメータ、異なる勾配規模
n_steps = 1000
np.random.seed(42)

# パラメータ1: 頻繁に大きな勾配
grads_param1 = np.random.randn(n_steps) * 10

# パラメータ2: まれに勾配（スパース）
grads_param2 = np.zeros(n_steps)
sparse_indices = np.random.choice(n_steps, size=50, replace=False)
grads_param2[sparse_indices] = np.random.randn(50) * 5

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(grads_param1, alpha=0.7)
axes[0].set_xlabel('ステップ')
axes[0].set_ylabel('勾配')
axes[0].set_title('パラメータ1: 頻繁な勾配')
axes[0].grid(True, alpha=0.3)

axes[1].plot(grads_param2, alpha=0.7)
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('勾配')
axes[1].set_title('パラメータ2: スパースな勾配')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【問題】")
print(f"パラメータ1の勾配RMS: {np.sqrt(np.mean(grads_param1**2)):.2f}")
print(f"パラメータ2の勾配RMS: {np.sqrt(np.mean(grads_param2**2)):.2f}")
print("")
print("同じ学習率では最適な更新ができない")

---

## 2. Adagrad

### 2.1 アイデア

**Adagrad** (Adaptive Gradient) は、過去の勾配の二乗和を追跡し、それで学習率を割ることでパラメータごとの学習率を調整します。

### 2.2 数式

$$
\begin{aligned}
G_t &= G_{t-1} + g_t^2 \\
\theta_{t+1} &= \theta_t - \frac{\eta}{\sqrt{G_t + \epsilon}} \cdot g_t
\end{aligned}
$$

- $G_t$: 過去の勾配の二乗の累積和
- $\epsilon$: ゼロ除算を防ぐ小さな値（例: $10^{-8}$）
- 勾配が大きいパラメータは学習率が自動的に小さくなる

In [ ]:
class Adagrad:
    """
    Adagrad optimizer
    
    G = G + g²
    θ = θ - η * g / √(G + ε)
    """
    
    def __init__(self, lr=0.01, eps=1e-8):
        self.lr = lr
        self.eps = eps
        self.G = None  # 勾配の二乗の累積
    
    def update(self, param, grad):
        if self.G is None:
            self.G = np.zeros_like(param)
        
        self.G += grad ** 2
        param -= self.lr * grad / (np.sqrt(self.G) + self.eps)
        
        return param


def adagrad_optimize(grad_fn, start, lr=0.5, n_steps=100, **kwargs):
    """Adagrad最適化"""
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    G = np.zeros_like(pos)
    eps = 1e-8
    
    for _ in range(n_steps):
        grad = grad_fn(pos[0], pos[1], **kwargs)
        G += grad ** 2
        pos = pos - lr * grad / (np.sqrt(G) + eps)
        path.append(pos.copy())
    
    return np.array(path)


print("Adagrad を定義しました")

In [ ]:
# Adagradの効果を可視化

def pathological_loss(x, y, condition_number=50):
    return x**2 + condition_number * y**2

def pathological_grad(x, y, condition_number=50):
    return np.array([2*x, 2*condition_number*y])


# SGD vs Adagrad
start = (3, 1)
n_steps = 100

# SGD
path_sgd = [np.array(start)]
pos = np.array(start, dtype=float)
for _ in range(n_steps):
    grad = pathological_grad(pos[0], pos[1])
    pos = pos - 0.01 * grad
    path_sgd.append(pos.copy())
path_sgd = np.array(path_sgd)

# Adagrad
path_adagrad = adagrad_optimize(pathological_grad, start, lr=0.5, n_steps=n_steps)

# 可視化
x = np.linspace(-4, 4, 200)
y = np.linspace(-2, 2, 200)
X, Y = np.meshgrid(x, y)
Z = pathological_loss(X, Y)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 軌跡
axes[0].contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.7)
axes[0].plot(path_sgd[:, 0], path_sgd[:, 1], 'r.-', markersize=2, linewidth=1, alpha=0.7, label='SGD')
axes[0].plot(path_adagrad[:, 0], path_adagrad[:, 1], 'b.-', markersize=2, linewidth=1, alpha=0.7, label='Adagrad')
axes[0].scatter([0], [0], color='green', s=100, marker='*', zorder=5)
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('SGD vs Adagrad')
axes[0].legend()
axes[0].set_xlim(-1, 4)
axes[0].set_ylim(-1.5, 1.5)

# 損失の推移
losses_sgd = [pathological_loss(p[0], p[1]) for p in path_sgd]
losses_adagrad = [pathological_loss(p[0], p[1]) for p in path_adagrad]

axes[1].semilogy(losses_sgd, 'r-', linewidth=2, label='SGD')
axes[1].semilogy(losses_adagrad, 'b-', linewidth=2, label='Adagrad')
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('損失（対数スケール）')
axes[1].set_title('収束速度の比較')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【Adagradの効果】")
print("- y方向: 勾配が大きい → 学習率が自動的に減少")
print("- x方向: 勾配が小さい → 相対的に大きな学習率を維持")

### 2.3 Adagradの問題点

**学習率が単調減少**: $G_t$ は累積なので増加し続け、学習率 $\eta / \sqrt{G_t}$ は減少し続ける

→ 学習の後半で学習率が小さくなりすぎ、収束が止まる

In [ ]:
# Adagradの学習率減衰問題

# シミュレーション: 一定の勾配を受け続けた場合
n_steps = 500
grad = 1.0  # 一定の勾配
G = 0
effective_lrs = []
lr = 0.5
eps = 1e-8

for t in range(1, n_steps + 1):
    G += grad ** 2
    effective_lr = lr / (np.sqrt(G) + eps)
    effective_lrs.append(effective_lr)

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(effective_lrs, 'b-', linewidth=2)
ax.set_xlabel('ステップ')
ax.set_ylabel('実効学習率')
ax.set_title('Adagradの学習率減衰')
ax.grid(True, alpha=0.3)

# 理論曲線
t = np.arange(1, n_steps + 1)
theoretical = lr / np.sqrt(t)
ax.plot(t, theoretical, 'r--', linewidth=2, label=r'$\eta / \sqrt{t}$')
ax.legend()

plt.tight_layout()
plt.show()

print(f"【問題】")
print(f"ステップ 1: 学習率 = {effective_lrs[0]:.4f}")
print(f"ステップ 100: 学習率 = {effective_lrs[99]:.4f}")
print(f"ステップ 500: 学習率 = {effective_lrs[-1]:.4f}")
print("")
print("学習が進むにつれて更新量が小さくなりすぎる")

---

## 3. RMSprop

### 3.1 アイデア

**RMSprop** (Root Mean Square Propagation) は、Adagradの問題を解決するために **指数移動平均** を使用します。

### 3.2 数式

$$
\begin{aligned}
v_t &= \rho \cdot v_{t-1} + (1 - \rho) \cdot g_t^2 \\
\theta_{t+1} &= \theta_t - \frac{\eta}{\sqrt{v_t + \epsilon}} \cdot g_t
\end{aligned}
$$

- $\rho$: 減衰率（通常 0.9 または 0.99）
- 指数移動平均により、古い勾配の影響が減衰

In [ ]:
class RMSprop:
    """
    RMSprop optimizer
    
    v = ρ * v + (1 - ρ) * g²
    θ = θ - η * g / √(v + ε)
    """
    
    def __init__(self, lr=0.01, rho=0.9, eps=1e-8):
        self.lr = lr
        self.rho = rho
        self.eps = eps
        self.v = None
    
    def update(self, param, grad):
        if self.v is None:
            self.v = np.zeros_like(param)
        
        self.v = self.rho * self.v + (1 - self.rho) * grad ** 2
        param -= self.lr * grad / (np.sqrt(self.v) + self.eps)
        
        return param


def rmsprop_optimize(grad_fn, start, lr=0.1, rho=0.9, n_steps=100, **kwargs):
    """RMSprop最適化"""
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    v = np.zeros_like(pos)
    eps = 1e-8
    
    for _ in range(n_steps):
        grad = grad_fn(pos[0], pos[1], **kwargs)
        v = rho * v + (1 - rho) * grad ** 2
        pos = pos - lr * grad / (np.sqrt(v) + eps)
        path.append(pos.copy())
    
    return np.array(path)


print("RMSprop を定義しました")

In [ ]:
# Adagrad vs RMSprop: 学習率の推移

n_steps = 500
grad = 1.0
lr = 0.5
eps = 1e-8
rho = 0.9

# Adagrad
G = 0
lrs_adagrad = []
for t in range(n_steps):
    G += grad ** 2
    lrs_adagrad.append(lr / (np.sqrt(G) + eps))

# RMSprop
v = 0
lrs_rmsprop = []
for t in range(n_steps):
    v = rho * v + (1 - rho) * grad ** 2
    lrs_rmsprop.append(lr / (np.sqrt(v) + eps))

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(lrs_adagrad, 'b-', linewidth=2, label='Adagrad')
ax.plot(lrs_rmsprop, 'r-', linewidth=2, label='RMSprop')
ax.set_xlabel('ステップ')
ax.set_ylabel('実効学習率')
ax.set_title('Adagrad vs RMSprop: 学習率の推移')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【RMSpropの利点】")
print("- 学習率が安定する（単調減少しない）")
print(f"- RMSpropの定常状態学習率: {lrs_rmsprop[-1]:.4f}")
print(f"- Adagradの最終学習率: {lrs_adagrad[-1]:.6f}")

---

## 4. Adam

### 4.1 アイデア

**Adam** (Adaptive Moment Estimation) は、Momentum と RMSprop を組み合わせた手法です。

- **1次モーメント** (m): 勾配の指数移動平均 → Momentumの効果
- **2次モーメント** (v): 勾配の二乗の指数移動平均 → RMSpropの効果

### 4.2 数式

$$
\begin{aligned}
m_t &= \beta_1 m_{t-1} + (1 - \beta_1) g_t \\
v_t &= \beta_2 v_{t-1} + (1 - \beta_2) g_t^2 \\
\hat{m}_t &= \frac{m_t}{1 - \beta_1^t} \quad \text{(バイアス補正)} \\
\hat{v}_t &= \frac{v_t}{1 - \beta_2^t} \quad \text{(バイアス補正)} \\
\theta_{t+1} &= \theta_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t
\end{aligned}
$$

デフォルト: $\beta_1 = 0.9$, $\beta_2 = 0.999$, $\epsilon = 10^{-8}$

In [ ]:
class Adam:
    """
    Adam optimizer
    
    m = β1 * m + (1 - β1) * g
    v = β2 * v + (1 - β2) * g²
    m_hat = m / (1 - β1^t)
    v_hat = v / (1 - β2^t)
    θ = θ - η * m_hat / (√v_hat + ε)
    """
    
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = None
        self.v = None
        self.t = 0
    
    def update(self, param, grad):
        if self.m is None:
            self.m = np.zeros_like(param)
            self.v = np.zeros_like(param)
        
        self.t += 1
        
        # 1次モーメント（勾配の平均）
        self.m = self.beta1 * self.m + (1 - self.beta1) * grad
        
        # 2次モーメント（勾配の二乗の平均）
        self.v = self.beta2 * self.v + (1 - self.beta2) * grad ** 2
        
        # バイアス補正
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)
        
        # パラメータ更新
        param -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)
        
        return param


def adam_optimize(grad_fn, start, lr=0.1, beta1=0.9, beta2=0.999, n_steps=100, **kwargs):
    """Adam最適化"""
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    m = np.zeros_like(pos)
    v = np.zeros_like(pos)
    eps = 1e-8
    
    for t in range(1, n_steps + 1):
        grad = grad_fn(pos[0], pos[1], **kwargs)
        
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * grad ** 2
        
        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        
        pos = pos - lr * m_hat / (np.sqrt(v_hat) + eps)
        path.append(pos.copy())
    
    return np.array(path)


print("Adam を定義しました")

### 4.3 バイアス補正の必要性

初期状態では $m_0 = v_0 = 0$ なので、最初の数ステップでは $m_t, v_t$ が過小評価されます。

In [ ]:
# バイアス補正の効果

n_steps = 50
true_mean = 1.0  # 真の勾配平均
beta1 = 0.9

m = 0
m_raw = []  # 補正なし
m_corrected = []  # 補正あり

for t in range(1, n_steps + 1):
    m = beta1 * m + (1 - beta1) * true_mean
    m_raw.append(m)
    m_corrected.append(m / (1 - beta1 ** t))

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(m_raw, 'b-', linewidth=2, label='補正なし m')
ax.plot(m_corrected, 'r-', linewidth=2, label='補正あり m / (1-β^t)')
ax.axhline(y=true_mean, color='green', linestyle='--', linewidth=2, label=f'真の平均 = {true_mean}')

ax.set_xlabel('ステップ')
ax.set_ylabel('m の推定値')
ax.set_title('Adamのバイアス補正')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"【バイアス補正の効果】")
print(f"ステップ 1:")
print(f"  補正なし: {m_raw[0]:.4f}")
print(f"  補正あり: {m_corrected[0]:.4f}")
print(f"  真の値:   {true_mean}")

In [ ]:
# 全手法の比較

start = (3, 1)
n_steps = 100

# 各手法での最適化
path_sgd = []
pos = np.array(start, dtype=float)
path_sgd.append(pos.copy())
for _ in range(n_steps):
    grad = pathological_grad(pos[0], pos[1])
    pos = pos - 0.01 * grad
    path_sgd.append(pos.copy())
path_sgd = np.array(path_sgd)

path_adagrad = adagrad_optimize(pathological_grad, start, lr=0.5, n_steps=n_steps)
path_rmsprop = rmsprop_optimize(pathological_grad, start, lr=0.1, rho=0.9, n_steps=n_steps)
path_adam = adam_optimize(pathological_grad, start, lr=0.1, n_steps=n_steps)

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 軌跡
axes[0].contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.7)
axes[0].plot(path_sgd[:, 0], path_sgd[:, 1], 'r.-', markersize=2, linewidth=1, alpha=0.7, label='SGD')
axes[0].plot(path_adagrad[:, 0], path_adagrad[:, 1], 'g.-', markersize=2, linewidth=1, alpha=0.7, label='Adagrad')
axes[0].plot(path_rmsprop[:, 0], path_rmsprop[:, 1], 'b.-', markersize=2, linewidth=1, alpha=0.7, label='RMSprop')
axes[0].plot(path_adam[:, 0], path_adam[:, 1], 'm.-', markersize=2, linewidth=1, alpha=0.7, label='Adam')
axes[0].scatter([0], [0], color='black', s=100, marker='*', zorder=5)
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('最適化手法の比較')
axes[0].legend()
axes[0].set_xlim(-1, 4)
axes[0].set_ylim(-1.5, 1.5)

# 損失の推移
losses = {
    'SGD': [pathological_loss(p[0], p[1]) for p in path_sgd],
    'Adagrad': [pathological_loss(p[0], p[1]) for p in path_adagrad],
    'RMSprop': [pathological_loss(p[0], p[1]) for p in path_rmsprop],
    'Adam': [pathological_loss(p[0], p[1]) for p in path_adam],
}

colors = {'SGD': 'r', 'Adagrad': 'g', 'RMSprop': 'b', 'Adam': 'm'}
for name, loss in losses.items():
    axes[1].semilogy(loss, color=colors[name], linewidth=2, label=name)

axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('損失（対数スケール）')
axes[1].set_title('収束速度の比較')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【最終損失】")
for name, loss in losses.items():
    print(f"{name:10s}: {loss[-1]:.6f}")

---

## 5. AdamW と Weight Decay

### 5.1 L2正則化 vs Weight Decay

**L2正則化**: 損失関数に $\frac{\lambda}{2} \|\theta\|^2$ を加える

**Weight Decay**: パラメータ更新時に $\theta \leftarrow (1 - \lambda) \theta$ を適用

SGDでは同等ですが、Adamでは異なる効果を持ちます。

In [ ]:
# L2正則化 vs Weight Decay の違い

print("【SGDの場合】")
print("")
print("L2正則化:")
print("  L' = L + (λ/2)||θ||²")
print("  ∇L' = ∇L + λθ")
print("  θ = θ - η(∇L + λθ) = θ - η∇L - ηλθ")
print("")
print("Weight Decay:")
print("  θ = θ - η∇L - ηλθ")
print("")
print("→ 両者は同等")
print("")
print("-" * 50)
print("")
print("【Adamの場合】")
print("")
print("L2正則化:")
print("  θ = θ - η * m_hat / √v_hat  (mとvは正則化項を含む勾配から計算)")
print("  → 正則化項も適応的にスケーリングされる")
print("")
print("Weight Decay (AdamW):")
print("  θ = θ - η * m_hat / √v_hat - ηλθ")
print("  → 正則化項は適応的スケーリングの影響を受けない")
print("")
print("→ AdamWの方が正則化効果が安定")

In [ ]:
class AdamW:
    """
    AdamW optimizer (Adam with decoupled Weight Decay)
    
    θ = θ - η * m_hat / (√v_hat + ε) - η * λ * θ
    """
    
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8, weight_decay=0.01):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.weight_decay = weight_decay
        self.m = None
        self.v = None
        self.t = 0
    
    def update(self, param, grad):
        if self.m is None:
            self.m = np.zeros_like(param)
            self.v = np.zeros_like(param)
        
        self.t += 1
        
        # 1次・2次モーメント
        self.m = self.beta1 * self.m + (1 - self.beta1) * grad
        self.v = self.beta2 * self.v + (1 - self.beta2) * grad ** 2
        
        # バイアス補正
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)
        
        # AdamW: Weight DecayをAdam更新とは別に適用
        param -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)
        param -= self.lr * self.weight_decay * param  # 分離されたWeight Decay
        
        return param


print("AdamW を定義しました")

---

## 6. 手法の比較

### 6.1 Rosenbrock関数での比較

In [ ]:
def rosenbrock(x, y):
    return (1 - x)**2 + 100 * (y - x**2)**2

def rosenbrock_grad(x, y):
    dx = -2 * (1 - x) - 400 * x * (y - x**2)
    dy = 200 * (y - x**2)
    return np.array([dx, dy])


# 各手法での最適化
start = (-1.5, 1.5)
n_steps = 2000

paths = {
    'SGD': [],
    'Adagrad': adagrad_optimize(rosenbrock_grad, start, lr=0.5, n_steps=n_steps),
    'RMSprop': rmsprop_optimize(rosenbrock_grad, start, lr=0.01, rho=0.9, n_steps=n_steps),
    'Adam': adam_optimize(rosenbrock_grad, start, lr=0.01, n_steps=n_steps),
}

# SGD
pos = np.array(start, dtype=float)
paths['SGD'].append(pos.copy())
for _ in range(n_steps):
    grad = rosenbrock_grad(pos[0], pos[1])
    pos = pos - 0.001 * grad
    paths['SGD'].append(pos.copy())
paths['SGD'] = np.array(paths['SGD'])

# 可視化
x = np.linspace(-2, 2, 200)
y = np.linspace(-1, 3, 200)
X_r, Y_r = np.meshgrid(x, y)
Z_r = rosenbrock(X_r, Y_r)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 軌跡
colors = {'SGD': 'r', 'Adagrad': 'g', 'RMSprop': 'b', 'Adam': 'm'}
axes[0].contour(X_r, Y_r, np.log(Z_r + 1), levels=30, cmap='viridis', alpha=0.7)

for name, path in paths.items():
    axes[0].plot(path[:, 0], path[:, 1], color=colors[name], linewidth=1, alpha=0.7, label=name)

axes[0].scatter([1], [1], color='black', s=100, marker='*', zorder=5, label='最適点')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('Rosenbrock関数での最適化')
axes[0].legend()

# 損失の推移
for name, path in paths.items():
    losses = [rosenbrock(p[0], p[1]) for p in path]
    axes[1].semilogy(losses, color=colors[name], linewidth=2, label=name)

axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('損失（対数スケール）')
axes[1].set_title('収束速度の比較')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【最終位置】")
for name, path in paths.items():
    dist = np.linalg.norm(path[-1] - np.array([1, 1]))
    print(f"{name:10s}: ({path[-1, 0]:.4f}, {path[-1, 1]:.4f}), 最適点からの距離: {dist:.4f}")

### 6.2 オプティマイザの特性まとめ

In [ ]:
# オプティマイザの特性比較表

print("="*80)
print("オプティマイザの特性比較")
print("="*80)
print("")
print(f"{'手法':<12} {'適応学習率':<10} {'モーメンタム':<12} {'メモリ':<10} {'推奨用途'}")
print("-"*80)
print(f"{'SGD':<12} {'×':<10} {'×':<12} {'θのみ':<10} {'凸問題、ファインチューニング'}")
print(f"{'Momentum':<12} {'×':<10} {'○':<12} {'θ, v':<10} {'一般的な学習'}")
print(f"{'Adagrad':<12} {'○':<10} {'×':<12} {'θ, G':<10} {'スパースデータ'}")
print(f"{'RMSprop':<12} {'○':<10} {'×':<12} {'θ, v':<10} {'RNN'}")
print(f"{'Adam':<12} {'○':<10} {'○':<12} {'θ, m, v':<10} {'一般的なデフォルト'}")
print(f"{'AdamW':<12} {'○':<10} {'○':<12} {'θ, m, v':<10} {'Transformer, ViT'}")
print("-"*80)
print("")
print("【デフォルトハイパーパラメータ】")
print("  Adam/AdamW: lr=0.001, β1=0.9, β2=0.999, ε=1e-8")
print("  RMSprop:    lr=0.01, ρ=0.9")
print("  Momentum:   lr=0.01, μ=0.9")

---

## 7. 演習問題

### 演習 7.1: Adadelta の実装

Adadeltaは学習率パラメータを必要としないオプティマイザです。実装してください。

$$
\begin{aligned}
E[g^2]_t &= \rho E[g^2]_{t-1} + (1-\rho) g_t^2 \\
\Delta\theta_t &= -\frac{\sqrt{E[\Delta\theta^2]_{t-1} + \epsilon}}{\sqrt{E[g^2]_t + \epsilon}} g_t \\
E[\Delta\theta^2]_t &= \rho E[\Delta\theta^2]_{t-1} + (1-\rho) \Delta\theta_t^2
\end{aligned}
$$

In [ ]:
# 演習 7.1: 解答欄

class Adadelta:
    def __init__(self, rho=0.9, eps=1e-8):
        # TODO: 実装
        pass
    
    def update(self, param, grad):
        # TODO: 実装
        pass

# TODO: テスト

### 演習 7.2: β1, β2 の影響

Adamのハイパーパラメータ β1, β2 を変えて、収束速度への影響を観察してください。

In [ ]:
# 演習 7.2: 解答欄

# TODO: β1 = [0.5, 0.9, 0.99], β2 = [0.9, 0.999, 0.9999] を比較

pass

### 演習 7.3: NAdam の実装

NAdam は Adam に Nesterov モーメンタムを組み合わせた手法です。実装してください。

In [ ]:
# 演習 7.3: 解答欄

class NAdam:
    # TODO: 実装
    pass

---

## 8. まとめと次のステップ

### このノートブックで学んだこと

1. **Adagrad**: 
   - 過去の勾配二乗和で学習率を調整
   - 問題: 学習率が単調減少

2. **RMSprop**:
   - 指数移動平均を使用して問題を解決
   - 安定した適応学習率

3. **Adam**:
   - Momentum + RMSprop
   - バイアス補正が重要
   - 最も広く使われるオプティマイザ

4. **AdamW**:
   - Weight Decayを分離
   - Transformerなどの大規模モデルで推奨

### 次のノートブック（113: 学習率スケジューリング）への橋渡し

適応学習率は便利ですが、以下の課題が残ります：

- **初期学習率の設定**: まだ手動で選ぶ必要がある
- **学習の進行に応じた調整**: Warmup, Decay などのスケジュール

次のノートブックでは、**学習率スケジューリング** を学びます。

---

## 参考文献

1. Duchi, J., et al. (2011). Adaptive subgradient methods for online learning and stochastic optimization. *JMLR*.
2. Tieleman, T., & Hinton, G. (2012). Lecture 6.5—RMSprop. *COURSERA: Neural Networks for Machine Learning*.
3. Kingma, D. P., & Ba, J. (2015). Adam: A method for stochastic optimization. *ICLR*.
4. Loshchilov, I., & Hutter, F. (2019). Decoupled weight decay regularization. *ICLR*.